In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("./local_chat_glm_6b_model", trust_remote_code=True)


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [2]:
# model = AutoModel.from_pretrained("./local_chat_glm_6b_model", trust_remote_code=True).half().cuda(1)
# model = AutoModel.from_pretrained("./local_chat_glm_6b_model", trust_remote_code=True).cuda()
model = AutoModel.from_pretrained("./local_chat_glm_6b_model", trust_remote_code=True).to("cpu", dtype=float)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# %%time
# response, history = model.chat(tokenizer, "你好", history=[])
# print(response)

# model
# model.transformer.num_layers
# print(model.transformer)
# model.transformer.eval()
model.transformer


ChatGLMModel(
  (word_embeddings): Embedding(130528, 4096)
  (layers): ModuleList(
    (0-27): 28 x GLMBlock(
      (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (attention): SelfAttention(
        (rotary_emb): RotaryEmbedding()
        (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
        (dense): Linear(in_features=4096, out_features=4096, bias=True)
      )
      (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (mlp): GLU(
        (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
        (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
      )
    )
  )
  (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
)

In [4]:
# %%time
# response, history = model.chat(tokenizer, "你好", history=[])
# print(response)


In [5]:
import torch

# model.transformer.layers
# type(model.transformer.word_embeddings)
# type(model.transformer)
# model.transformer.eval()

import numpy as np
input_ids = torch.tensor(np.array([[5, 74874, 130001, 130004]]), dtype=torch.long)

# switch model to inference mode
model.transformer.eval()
onnx_path = "./output_onnx"

# disable gradients calculation for reducing memory consumption
with torch.no_grad():
    # export model to ONNX format
    torch.onnx.export(
        model.transformer,  # model instance
        input_ids,  # inputs for model tracing
        onnx_path,  # output file for saving result
        input_names=['tokens'],  # model input name for onnx representation
        output_names=['last_hidden_state', 'pooler_out'],  # model output names for onnx representation
    )
print('Text Encoder successfully converted to ONNX')


------------------------forword--->
input_ids.shape= torch.Size([1, 4])
inputs_embeds= None
input_ids.shape= torch.Size([1, 4])
batch_size= tensor(1)
seq_length= tensor(4)


/home/xiping_dev/.cache/huggingface/modules/transformers_modules/local_chat_glm_6b_model/modeling_chatglm.py:685: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  context_lengths = [seq.tolist().index(self.config.bos_token_id) for seq in input_ids]
/home/xiping_dev/.cache/huggingface/modules/transformers_modules/local_chat_glm_6b_model/modeling_chatglm.py:685: TracerWarning: Converting a tensor to a Python list might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  context_lengths = [seq.tolist().index(self.config.bos_token_id) for seq in input_ids]
/home/xiping_dev/.cache/huggingface/modules/transformers_modules/local_chat_glm_6b_model/modeling_chatglm.py:9

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Text Encoder successfully converted to ONNX


In [6]:
%%time
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

The dtype of attention mask (torch.int64) is not bool


inputs= {'input_ids': tensor([[     5,  74874, 130001, 130004]])}
self.device= cpu
inputs[input_ids]= tensor([[     5,  74874, 130001, 130004]])
input_ids.shape= torch.Size([1, 4])
batch_size= 1
seq_length= 4
------------------------forword--->
input_ids.shape= torch.Size([1, 4])
inputs_embeds= None
------------------------forword--->
input_ids.shape= torch.Size([1, 1])
inputs_embeds= None
------------------------forword--->
input_ids.shape= torch.Size([1, 1])
inputs_embeds= None
------------------------forword--->
input_ids.shape= torch.Size([1, 1])
inputs_embeds= None
------------------------forword--->
input_ids.shape= torch.Size([1, 1])
inputs_embeds= None
------------------------forword--->
input_ids.shape= torch.Size([1, 1])
inputs_embeds= None
------------------------forword--->
input_ids.shape= torch.Size([1, 1])
inputs_embeds= None
------------------------forword--->
input_ids.shape= torch.Size([1, 1])
inputs_embeds= None
------------------------forword--->
input_ids.shape= to

In [7]:
# !python -m transformers.onnx --model="./local_chat_glm_6b_model" onnx/ --framework pt

In [ ]:
onnx_path = "./output_onnx"
import onnx
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)


In [8]:
# from pathlib import Path

# ov_model_dir = Path("ov_model_dir")
# ov_model_dir.mkdir(exist_ok=True)

In [9]:
# %%time

# import gc
# import torch

# MODEL_1_ONNX_PATH = ov_model_dir / 'model_1.onnx'
# MODEL_1_OV_PATH = MODEL_1_ONNX_PATH.with_suffix('.xml')

# def convert_model_1_onnx(model_1: torch.nn.Module, onnx_path:Path):
    
#     if not onnx_path.exists():
#         input_ids = torch.ones((1, 77), dtype=torch.long)
#         # switch model to inference mode
#         model_1.eval()

#         # disable gradients calculation for reducing memory consumption
#         with torch.no_grad():
#             # export model to ONNX format
#             torch.onnx._export(
#                 text_encoder,  # model instance
#                 input_ids,  # inputs for model tracing
#                 onnx_path,  # output file for saving result
#                 input_names=['tokens'],  # model input name for onnx representation
#                 output_names=['last_hidden_state', 'pooler_out'],  # model output names for onnx representation
#                 opset_version=14,  # onnx opset version for export,
#                 onnx_shape_inference=False
#             )
#         print('Text Encoder successfully converted to ONNX')


# if not MODEL_1_OV_PATH.exists():
#     convert_model_1_onnx(text_encoder, MODEL_1_ONNX_PATH)
#     !mo --input_model $TEXT_ENCODER_ONNX_PATH --compress_to_fp16 --output_dir $sd2_1_model_dir
#     print('Text Encoder successfully converted to IR')
# else:
#     print(f"Text encoder will be loaded from {TEXT_ENCODER_OV_PATH}")

# del text_encoder

# gc.collect();

NameError: name 'text_encoder' is not defined